In [1]:
library(dada2); packageVersion("dada2")
library(ShortRead); packageVersion("ShortRead")
library(ggplot2); packageVersion("ggplot2")

Loading required package: Rcpp


[1] ‘1.1.8’

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min

Loading required package: BiocParallel
Loading required package: Biostrings
Loading required package: S4Vectors
Loading required pack

[1] ‘1.32.0’

[1] ‘2.1.0’

In [4]:
p1 <- "data/Understanding_methane_cycling_through_manipulation_of_synthetic_methane-oxidizing_communities__Synthetic_methane-oxidizing_com_itags_pl_1_[Synmetcoitagspl1_FD]/Raw_Data"
p2 <- "data/Understanding_methane_cycling_through_manipulation_of_synthetic_methane-oxidizing_communities__Synthetic_methane-oxidizing_com_itags_pl_2_[Synmetcoitagspl2_FD]/Raw_Data"
fns <- c(list.files(p1, full.names=T), list.files(p2, full.names=T))
head(fns)

In [34]:
fastqs <- fns[grepl(".fastq.gz", fns)]
fastqs <- sort(fastqs) # Sort ensures forward/reverse reads are in same order
fnFs <- fastqs[grepl("forward", fastqs)] # Just the forward read files
fnRs <- fastqs[grepl("reverse", fastqs)] # Just the reverse read files
# Get sample names from the first part of the forward read filenames
sample.names <- sapply(strsplit(fnFs, "_"), `[`, 1)
# Fully specify the path for the fnFs and fnRs
fnFs <- file.path(path, fnFs)
fnRs <- file.path(path, fnRs)

ERROR: Error in as.vector(x, "character"): cannot coerce type 'closure' to vector of type 'character'


In [ ]:
strsplit(fnFs, ".", fixed=T)